# Exam 2022/23

*This exam formed part of the assessment of this module that students took in 2022/23. You may wish to use it to help in your revision. There are no solutions available.*

This exam is worth 60% of the credit of the module. Solutions should be submitted to Gradescope.

## Linear systems of equations

### 1. Gauss-Seidel [4 marks]

Consider the linear system of equations given by

$$
 \begin{aligned}
 4 x_1 + 2 x_2 & = 0 \\
 x_1 + 4 x_2 & = -1.
 \end{aligned}
$$

Apply two iterations of the Gauss-Seidel method starting for $\vec{x}^{(0)} = (0, -1)^T$. Give the second component of your solution to 3 decimal places.

### 2. Comparison of methods [ 12 marks ]

The following tables show results from using three different iterative methods to solve a systems of linear equations which share a common structure. Table headers are:

 - `n`: the number of unknowns in the problem
 - `res`: final residual when computations stopped
 - `it`: the number of iterations required
 - `time`: time taken in secdonds

Comment on which is the best method for this problem and why by considering the accuacy and efficency of each method.

In [1]:
import pandas as pd

jacobi_table = pd.DataFrame(
    {
        "n": [10, 20, 40, 80, 160],
        "res": [1.12001e-06, 5.06838e-06, 2.145e-05, 8.81789e-05, 0.000357513],
        "it": [5974, 25224, 99298, 378258, 1414194],
        "time": [0.0167733, 0.0802511, 0.422201, 2.54317, 17.4071],
    }
)

cg_table = pd.DataFrame(
    {
        "n": [10, 20, 40, 80, 160],
        "res": [0.0, 1.33472e-14, 5.43684e-14, 5.79168e-13, 1.85349e-12],
        "it": [67, 122, 232, 452, 892],
        "time": [0.000360964, 0.000723931, 0.00158258, 0.00389178, 0.0119214],
    }
)

gmres_table = pd.DataFrame(
    {
        "n": [10, 20, 40, 80, 160],
        "res": [4.15407e-15, 4.4e-14, 2.27071e-13, 8.6665e-05, 0.0196501],
        "it": [67, 122, 232, 7580, 31823],
        "time": [0.00027179, 0.00050491, 0.0011734, 0.0579562, 0.434776],
    }
)

tables = {
    "Method A": jacobi_table,
    "Method B": cg_table,
    "Method C": gmres_table,
    }

for label, table in tables.items():
    display(f"{label}")
    display(table)

'Method A'

,n,res,it,time
0,10,0.000001,5974,0.016773
1,20,0.000005,25224,0.080251
2,40,0.000021,99298,0.422201
3,80,0.000088,378258,2.543170
4,160,0.000358,1414194,17.407100


'Method B'

,n,res,it,time
0,10,0.000000e+00,67,0.000361
1,20,1.334720e-14,122,0.000724
2,40,5.436840e-14,232,0.001583
3,80,5.791680e-13,452,0.003892
4,160,1.853490e-12,892,0.011921


'Method C'

,n,res,it,time
0,10,4.154070e-15,67,0.000272
1,20,4.400000e-14,122,0.000505
2,40,2.270710e-13,232,0.001173
3,80,8.666500e-05,7580,0.057956
4,160,1.965010e-02,31823,0.434776


## Time stepping methods

### 3. Predictor corrector methods [ 10 marks ]

In the notes you are given a method to construct a second order method, the midpoint method, by leveraging Euler's method. This question presents an alternative way to achieve higher order methods using the "predictor-corrector" method.

We consider a generic ordinary differential equation

$$
y'(t) = f(t, y), \quad \text{subject to} \quad y(0) = y_0.
$$

a. In this approach, we take one step of the original method and then do some averaging to get the final result.
   For example for Euler's method at each time step we perform:
       
   $$
   \begin{aligned}
   y^p & = y^{(i)} + \mathrm{d}t f(t^{(i)}, y^{(i)}) \\
   y^{(i+1)} & =  y^{(i)} + \mathrm{d}t/2 (f(t^{(i)}, y^{(i)}) + f(t^{(i+1)}, y^p)).
   \end{aligned}
   $$
       
For the example $f(t, y) = 1-y^2$ take **two steps** of the improved Euler method to compute the solution at $t = 0.5$ starting from $y(0) = 0$. Give your answer to 3 decimal places. [ 5 marks ]

b. We consider a similar approach for the midpoint method. This results in the following scheme:

   $$
   \begin{aligned}
   k_1 & = f(t^{(i)}, y^{(i)}) \\
   k_2 & = f(t^{(i)} + \mathrm{d}t/2, y^{(i)} + \mathrm{dt} k_1/2) \\
   k_3 & = f(t^{(i)} + \mathrm{d}t/2, y^{(i)} + \mathrm{dt} k_2/2) \\
   k_4 & = f(t^{(i)} + \mathrm{d}t, y^{(i)} + \mathrm{dt} k_3) \\
   y^{(i+1)} &= y^{(i)} + \mathrm{d}t/6 (k_1 + 2 k_2 + 2 k_3 + k_4).
   \end{aligned}
   $$
   
   For the example $f(t, y) = 1-y^2$ take **one step** of the improved midpoint method to compute the solution at $t = 0.5$ starting from $y(0) = 0$. Give your answer to 3 decimal places. [ 5 marks ]

c. A Python imlpementation of each method has been run and have produced the following error tables. The headings are:

   - `dt`: the time step
   - `y(1.0)`: the estimate of the solution at the final time
   - `Error`: the absolute error of the method against a known exact solution
   - `feval`: the number of right-hand side evaluations taken.
   
   What is the order of each method?

In [2]:
import numpy as np
from collections import OrderedDict

def euler_pc(y0, dt, T):
    y = [y0]
    
    t = [0]
    while t[-1] < T:
        new_t = t[-1] + dt
        
        pre = f(t[-1], y[-1])
        cor = f(t[-1] + dt, y[-1] + dt * pre)
        
        new_y = y[-1] + dt * (pre + cor)/2
        
        y.append(new_y)
        t.append(new_t)
    
    return y, t

def midpoint_pc(y0, dt, T):
    y = [y0]
    
    t = [0]
    while t[-1] < T:
        new_t = t[-1] + dt
        
        pre1 = f(t[-1], y[-1])
        pre = f(t[-1] + dt/2, y[-1]+dt*pre1/2)
        
        cor1 = f(t[-1] + dt/2, y[-1]+dt*pre/2)
        cor = f(t[-1] + dt, y[-1] + dt * cor1)
        
        new_y = y[-1] + dt * (pre1 + 2*pre + 2*cor1 + cor)/6
        
        y.append(new_y)
        t.append(new_t)
    
    return y, t

def exact(t):
    return np.sin(t)

feval = 0
def f(t, y):
    global feval
    feval += 1
    return np.cos(t)

def comparse_timestepping_methods(method, dt_range):
    global feval

    T = 1.0
    data = []

    for dt in dt_range:
        feval = 0
        y, t = method(exact(0.0), dt, T)
        error = abs(y[-1] - exact(T))

        row_data = OrderedDict({"dt": dt, "y(1.0)": y[-1], "Error": error, "feval": feval})
        data.append(row_data)
    
    return pd.DataFrame(data)

dt_range = [2**-j for j in range(1, 6)]

tables = {
    "Improved Euler": comparse_timestepping_methods(euler_pc, dt_range),
    "Improved midpoint": comparse_timestepping_methods(midpoint_pc, dt_range),
}

for label, table in tables.items():
    display(label)
    display(table)

'Improved Euler'

,dt,y(1.0),Error,feval
0,0.50000,0.823867,0.017604,4
1,0.25000,0.837084,0.004387,8
2,0.12500,0.840375,0.001096,16
3,0.06250,0.841197,0.000274,32
4,0.03125,0.841403,0.000068,64


'Improved midpoint'

,dt,y(1.0),Error,feval
0,0.50000,0.841489,1.839786e-05,8
1,0.25000,0.841472,1.143445e-06,16
2,0.12500,0.841471,7.136556e-08,32
3,0.06250,0.841471,4.458792e-09,64
4,0.03125,0.841471,2.786504e-10,128


d. Consider what you already know about the Euler and midpoint's method. Are these two methods actually an improvement? You should consider the accuracy and efficiency of each method.

## Nonlinear solvers

### 4. Applying nonlinear solvers [ 16 marks ]

Consider the nonlinear problem

$$
f(x) = x^4 - 3 x - 10 = 0.
$$
    
Take two steps of each method starting from the values given below to find a root. What is your estimate of the root? If the method breaks down please write "FAIL" in the answer box. Give your answer to 3 decimal places. [ 4 marks each ]

You may use that

$$
f'(x) = 4 x^3 - 3.
$$
   
   | Test | Method          | Starting values            |
   |------|-----------------|----------------------------|
   | A    | bisction        | $x_L = -2, x_R = 3$        |
   | B    | bisection       | $x_L = 0, x_R = 3$         |
   | C    | modified Newton | $x^{(0)} = 1.5$            |
   | D    | secant          | $x^{(0)} = 0, x^{(1)} = 1$ |

### 5. Visualising nonlinear solvers [ 8 marks ]

Consider the following plots of different iterative methods for solving nonlinear equations. Identify which method is being used in each image. [ 2 marks each ]

Image A:

![Image A](../../_static/img/exam_22/nonlinear1.svg)

Image B:

![Image B](../../_static/img/exam_22/nonlinear2.svg)

Image C:

![Image C](../../_static/img/exam_22/nonlinear3.svg)

Image D:

![Image D](../../_static/img/exam_22/nonlinear4.svg)

## Least squares fitting

### 6. Straight line predictions [ 5 marks ]

Find the straight line of best fit to the data set given by:

   | t   |   y  |
   |-----|------|
   | 0.0 | 18.0 |
   | 0.5 | 33.0 |
   | 1.0 | 43.0 |
   
What does your line predict for $t=2$? Give your answer to 3 decimal places.

## Summary [4 marks]

7. Choose an algorithm from the course.
   Give an application of this method to a real world problem.
   Do you think this method would be well suited to this application?